In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

print('Files in Drive:')
!ls /content/drive/

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
Files in Drive:
Colab Notebooks  created.txt  data  Louis共享文件夹  M.Sc-DIRO-UdeM  test.ods


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#%%
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import gzip


batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

import os
os.chdir("/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1")
# os.chdir("/Users/louis/Google Drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1")
# os.chdir("/Users/fanxiao/Google Drive/UdeM/IFT6135 Representation Learning/homework1/programming part ")
print(os.getcwd())



/content/drive/M.Sc-DIRO-UdeM/IFT6285-Traitements automatique des langues naturelles/TP1/ift6285-tp1


In [9]:

#%%
# Vectorize the data.

input_characters = set()
target_characters = set()
def loadData(corpuspath,size=None):
    input_texts = []
    target_texts = []
    with gzip.open(corpuspath, 'rt', encoding='ISO-8859-1') as f:
        lines = f.read().split('\n')
    input_phrase=[]
    target_phrase=[]
    # for line in lines[: min(num_samples, len(lines) - 1)]:
    i=0
    for line in lines:
        if not line.startswith('#begin') and not line.startswith('#end') and len(line.split('\t'))>1:
            line=line.encode("ascii", errors="ignore").decode()
            target_word, input_word = line.split('\t')
            input_word=input_word.strip().lower()
            target_word=target_word.strip().lower()
            input_phrase.append(input_word)
            target_phrase.append(target_word)
            input_phrase.append(' ')
            target_phrase.append(' ')
            if input_word=='.':
                # We use "tab" as the "start sequence" character
                # for the targets, and "\n" as "end sequence" character.
                input_texts.append(''.join(input_phrase))
                target_texts.append('\t'+''.join(target_phrase) + '\n')
                input_phrase=[]
                target_phrase=[]
                i+=1

            for char in input_word:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_word:
                if char not in target_characters:
                    target_characters.add(char)
            if size is not None and i>size: break
            
    return input_texts,target_texts
    
input_texts,target_texts=loadData('data/train-1544.gz',5500)
size_train=len(input_texts)
test_input_texts,test_target_texts=loadData('data/test-2834.gz',2000)
size_test=len(test_input_texts)


input_texts=np.array(input_texts)
target_texts=np.array(target_texts)
# np.random.seed(456)
# indexs = np.arange(0,len(input_texts))
# np.random.shuffle(indexs)
# input_texts = input_texts[indexs[0:size_train]]
# target_texts=target_texts[indexs[0:size_train]]

test_input_texts=np.array(test_input_texts)
test_target_texts=np.array(test_target_texts)

#input_texts,target_texts: list of phrases
#input_token_index: key-values of char
#encoder_input_data:3dimension, d1=phrase,d2=char index in phrase,d3=index of list of candidate chars
# print(input_texts[0:2])
# print(target_texts[0:2])

input_characters.add(' ')
target_characters.add(' ')
target_characters.add('\t')
target_characters.add('\n')
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of sentence samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print('sentence of training data:', size_train)
print('sentence of test data:', size_test)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# print(input_token_index)
# print(target_token_index)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


Number of sentence samples: 5501
Number of unique input tokens: 60
Number of unique output tokens: 62
Max sequence length for inputs: 817
Max sequence length for outputs: 841
sentence of training data: 5501
sentence of test data: 2001


In [0]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [0]:

#%%
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
# model.save('s2s.h5')


Train on 4400 samples, validate on 1101 samples
Epoch 1/100
4400/4400 [==============================] - 194s 44ms/step - loss: 0.4160 - val_loss: 0.4815
Epoch 2/100
4400/4400 [==============================] - 194s 44ms/step - loss: 0.3825 - val_loss: 0.4397
Epoch 3/100
4400/4400 [==============================] - 195s 44ms/step - loss: 0.3476 - val_loss: 0.4045
Epoch 4/100
4400/4400 [==============================] - 194s 44ms/step - loss: 0.3247 - val_loss: 0.3864
Epoch 5/100
  64/4400 [..............................] - ETA: 2:56 - loss: 0.3496

In [0]:
print(os.getcwd())
from keras.models import load_model
model.save('model1-lstm-4016samples-100epochs.h5')
# del model  # deletes the existing model

In [0]:

#%%
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence
  

#%%
'''
Functions of Evalutate the prediction
'''
def count_accuracy(pred_sents,target_sents):
    count_accu=0
    total=0
    for pred_sent,target_sent in zip(pred_sents,target_sents):
        pred_list=re.split(r"-| |\?",pred_sent)
        # pred_list=pred_sent.split(' ')
        for pred_token,target_token in zip(pred_list,target_sent):
            total+=1
            if pred_token==target_token.text:
                count_accu+=1
    return count_accu, total


In [10]:

#%%
print('-------predict train data:')
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-------predict train data:
-
Input sentence:  he also have a brother who be a well-respected professor and glaciologist , and a sister ( now decease ) who be also a scientist and a professor . 
Decoded sentence:  he was a september 1960 and 1998 , was deserviced as a second region and on the second red sox . 

-
Input sentence:  in 1977 environmental group present the petition to parliament carry 341,160 signature . 
Decoded sentence:  in 1981 , the population of the post of the mountain to around 2011 in a season of the mountain to also known as the destroy records . 

-
Input sentence:  it be threaten by habitat loss . 
Decoded sentence:  it is threatened by habitat loss . 

-
Input sentence:  abutilon eremitopetalum , commonly know as the hidden-petaled abutilon or hiddenpetal indian mallow , be a species of flowering shrub in the mallow family , malvaceae . 
Decoded sentence:  a star the community consists of the south community of the united states , and the first community . 

-


In [0]:
print(os.getcwd())
from keras.models import load_model
model.load('model1-lstm-4016samples-100epochs.h5')

In [0]:

#%%
print('-------predict train data:')
train_pred_sents=[]
for seq_index in range(500):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-- No. ',seq_index)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    train_pred_sents.append(decoded_sentence)

train_acc_count,count_total=count_accuracy(train_pred_sents,input_texts)
print('Accuracy on LSTM1 predicteur, train data:', train_acc_count,'/', count_total,'=',train_acc_count/count_total)





In [0]:

test_encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# test_decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')
# test_decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (test_input_text, test_target_text) in enumerate(zip(test_input_texts, test_target_texts)):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token_index[char]] = 1.
#     for t, char in enumerate(test_target_text):
#         # decoder_target_data is ahead of decoder_input_data by one timestep
#         test_decoder_input_data[i, t, target_token_index[char]] = 1.
#         if t > 0:
#             # decoder_target_data will be ahead by one timestep
#             # and will not include the start character.
#             test_decoder_target_data[i, t - 1, target_token_index[char]] = 1.


print('-----predict test data:')
test_pred_sents=[]
for seq_index in range(500):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = test_encoder_input_data[seq_index]
    decoded_sentence = decode_sequence(input_seq)
    print('-- No. ',seq_index)
    print('Input test sentence:', test_input_texts[seq_index])
    print('Decoded test sentence:', decoded_sentence)
    test_pred_sents.append(decoded_sentence)

test_acc_count,count_total=count_accuracy(test_pred_sents,test_input_texts)
print('Accuracy on LSTM1 predicteur, test data:', test_acc_count,'/', count_total,'=',test_acc_count/count_total)



